<a href="https://colab.research.google.com/github/panda1835/youtube-hunting/blob/main/03-Data-Annotation/01_synthesize-comment-annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download utilized scripts
!wget https://raw.githubusercontent.com/panda1835/youtube-hunting/main/Utils/utils.py

In [2]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
import importlib
from time import sleep

from google.colab import data_table

# import utilized scripts
import utils as utils

In [3]:
importlib.reload(utils)

<module 'utils' from '/content/utils.py'>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
root = "/content/drive/My Drive/Let-Hunt-Youtube/"
config_file_path = os.path.join(root, 'config.json')

config = json.load(open(config_file_path))

database_save_folder = config['folder']['database']
comment_save_folder = config['folder']['comment']
credentials_folder = config['folder']['credentials']

credentials_file_name = config['file']['comment-annotation-spreadsheet-credentials']
comment_annotation_db_file_name = config['database']['comment_50_annotation_synthesis']
comment_annotation_finalized_db_file_name = config['database']['comment_50_annotation_finalized']

credentials_file_path = os.path.join(root, credentials_folder, credentials_file_name)
comment_annotation_db_file_path = os.path.join(root, comment_save_folder, comment_annotation_db_file_name)
comment_annotation_finalized_db_file_path = os.path.join(root, comment_save_folder, comment_annotation_finalized_db_file_name)

In [6]:
# get all 50 annotation sheets

import csv
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ref: https://stackoverflow.com/questions/3287651/download-a-spreadsheet-from-google-docs-using-python

scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file_path, scope)

annotation_spreadsheets = [
    "1FhekVTIujNIfG1VDG0zJqPWqfqo7oG2u3mNHJzlvKfg", # 01-10
    "1lHuszfSyDKt9O2tlb_bWeb9tlpAAi6JJKd9EsmqayPg", # 11-20
    "13T56J69Av4jeA-I0QFfaQLif_yWzh-77duvbgnoneeg", # 21-30
    "1lWoo_-wgjdncAOXMYV0OTu_4f5h2AbfJjkGZm1Jb9I8", # 31-40
    "1TtzKwXHC-BW9Iazt5Hi4kY5GIKIEKM2oDIgzmpRjYAA"  # 41-50
]

client = gspread.authorize(credentials)

file_list = []

for spreadsheet_id in annotation_spreadsheets:
  spreadsheet = client.open_by_key(spreadsheet_id)
  for worksheet in spreadsheet.worksheets():
      filename = worksheet.title + '.csv'
      file_list.append(filename)
      with open(filename, 'w') as f:
          writer = csv.writer(f)
          writer.writerows(worksheet.get_all_values())

In [10]:
annotation_data = pd.DataFrame()
for annotation_file in file_list:
  if annotation_file != 'category.csv':
    annotation_data = annotation_data.append(pd.read_csv(annotation_file), ignore_index=True)

annotation_data = annotation_data.drop(["Unnamed: 10", "Unnamed: 13"], axis=1)

utils.display_table(annotation_data)

,author_channel_id,published_time,comment_id,video_id,author_display_name,parent_comment_id,num_like,content,An,Note,Unnamed: 10,Linh,Note.1,Unnamed: 13,Ngoc,Note.2
0,UCAihGaM9aBrWjzHQLVSu6lw,2021-10-21T11:54:54Z,UgyrZyHpEBwjTaHZjbB4AaABAg,wQiJKW13eJ0,Truc Tran,NaN,0,"May xao lon quá , hàng này nó không có lừa , n...",\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN
1,UCdYC68ahn-aSrxu4o8sDgWQ,2021-10-20T08:02:47Z,UgxNTCsCM4ShZva34VN4AaABAg,wQiJKW13eJ0,Van Day Bui,NaN,0,Bao huy tín nhe ae. Yên tâm,shop feedback,NaN,NaN,shop feedback,NaN,NaN,\ngreetings / networking,NaN
2,UCDJuo0gNOMXR_KeuX6oJ-fw,2021-10-19T13:20:57Z,UgzOHjHA4MVy1hS_-454AaABAg,wQiJKW13eJ0,Hùng Đỗ,NaN,0,Mấy cây lục mini độc quá 😋,the rifle,NaN,NaN,the rifle,NaN,NaN,the rifle,NaN
3,UCNM0F7D4XZXpZsbqNXnLKzw,2021-10-19T12:06:46Z,UgwjKxD6f69xEi2B1sB4AaABAg,wQiJKW13eJ0,Tuan Thanh,NaN,0,Khoái khoái. Chảy nước miếng chảy nước miếng.kk,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN
4,UCqkdf0_F1Vxa01EulcPjQ0g,2021-10-19T12:19:53Z,UgwjKxD6f69xEi2B1sB4AaABAg.9TfvPQaztri9TfwuWzdbuu,NaN,cuộc sống vùng quê pcp,UgwjKxD6f69xEi2B1sB4AaABAg,0,Kkk haha,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,UCSs1OivW-WcBJRQ50oCPiBQ,2021-07-09T00:46:30Z,Ugzv0KGLrGqNp4GrGOB4AaABAg,Wg8sj16vj3w,Phương Phương,NaN,1,Ok,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,unclear,NaN
956,UC7VycIgXoMhKSVjQJ3S8M0w,2021-07-08T15:08:41Z,Ugzvpuek-3kHMyR-JqN4AaABAg,Wg8sj16vj3w,Phuong Vi,NaN,1,Chào shop,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN
957,UCAOv906DWPsM1WRy2jjRLtQ,2021-07-08T14:04:39Z,UgzGBGIOuWDZf_RkZzt4AaABAg,Wg8sj16vj3w,Minh Nguyễn Văn,NaN,2,Ok,\ngreetings / networking,NaN,NaN,\ngreetings / networking,NaN,NaN,unclear,NaN
958,UCkJyJ0SfvipWs_tKZQ3w92A,2019-10-31T23:14:14Z,Ugz5Jmg7C7yPZEBMkIJ4AaABAg,UBEj0WuN-T0,Hot Girl Vlog,NaN,1,Cái này mà có cơm lam nữa thì khỏi chê,\ngreetings / networking,NaN,NaN,target animal info,NaN,NaN,target animal info,chắc ăn với con mới săn


In [14]:
# save database
assert annotation_data.comment_id.nunique() == len(annotation_data), print('duplicated rows')
utils.save_database(annotation_data, comment_annotation_db_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Comments/COMMENT_50_ANNOTATION.csv


In [19]:
mode = annotation_data[['An', 'Linh', 'Ngoc']].mode(axis=1)
annotation_data['finalized_tag'] = np.where(mode.isna().any(1), mode[0], 'split')

In [21]:
utils.display_table(annotation_data)

,author_channel_id,published_time,comment_id,video_id,author_display_name,parent_comment_id,num_like,content,An,Note,Linh,Note.1,Ngoc,Note.2,finalized_tag
0,UCAihGaM9aBrWjzHQLVSu6lw,2021-10-21T11:54:54Z,UgyrZyHpEBwjTaHZjbB4AaABAg,wQiJKW13eJ0,Truc Tran,NaN,0,"May xao lon quá , hàng này nó không có lừa , n...",\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking
1,UCdYC68ahn-aSrxu4o8sDgWQ,2021-10-20T08:02:47Z,UgxNTCsCM4ShZva34VN4AaABAg,wQiJKW13eJ0,Van Day Bui,NaN,0,Bao huy tín nhe ae. Yên tâm,shop feedback,NaN,shop feedback,NaN,\ngreetings / networking,NaN,shop feedback
2,UCDJuo0gNOMXR_KeuX6oJ-fw,2021-10-19T13:20:57Z,UgzOHjHA4MVy1hS_-454AaABAg,wQiJKW13eJ0,Hùng Đỗ,NaN,0,Mấy cây lục mini độc quá 😋,the rifle,NaN,the rifle,NaN,the rifle,NaN,the rifle
3,UCNM0F7D4XZXpZsbqNXnLKzw,2021-10-19T12:06:46Z,UgwjKxD6f69xEi2B1sB4AaABAg,wQiJKW13eJ0,Tuan Thanh,NaN,0,Khoái khoái. Chảy nước miếng chảy nước miếng.kk,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking
4,UCqkdf0_F1Vxa01EulcPjQ0g,2021-10-19T12:19:53Z,UgwjKxD6f69xEi2B1sB4AaABAg.9TfvPQaztri9TfwuWzdbuu,NaN,cuộc sống vùng quê pcp,UgwjKxD6f69xEi2B1sB4AaABAg,0,Kkk haha,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,UCSs1OivW-WcBJRQ50oCPiBQ,2021-07-09T00:46:30Z,Ugzv0KGLrGqNp4GrGOB4AaABAg,Wg8sj16vj3w,Phương Phương,NaN,1,Ok,\ngreetings / networking,NaN,\ngreetings / networking,NaN,unclear,NaN,\ngreetings / networking
956,UC7VycIgXoMhKSVjQJ3S8M0w,2021-07-08T15:08:41Z,Ugzvpuek-3kHMyR-JqN4AaABAg,Wg8sj16vj3w,Phuong Vi,NaN,1,Chào shop,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking,NaN,\ngreetings / networking
957,UCAOv906DWPsM1WRy2jjRLtQ,2021-07-08T14:04:39Z,UgzGBGIOuWDZf_RkZzt4AaABAg,Wg8sj16vj3w,Minh Nguyễn Văn,NaN,2,Ok,\ngreetings / networking,NaN,\ngreetings / networking,NaN,unclear,NaN,\ngreetings / networking
958,UCkJyJ0SfvipWs_tKZQ3w92A,2019-10-31T23:14:14Z,Ugz5Jmg7C7yPZEBMkIJ4AaABAg,UBEj0WuN-T0,Hot Girl Vlog,NaN,1,Cái này mà có cơm lam nữa thì khỏi chê,\ngreetings / networking,NaN,target animal info,NaN,target animal info,chắc ăn với con mới săn,target animal info


In [23]:
# save database
assert annotation_data.comment_id.nunique() == len(annotation_data), print('duplicated rows')
utils.save_database(annotation_data, comment_annotation_finalized_db_file_path)

Database is saved to /content/drive/My Drive/Let-Hunt-Youtube/Comments/COMMENT_50_ANNOTATION_FINALIZED.csv
